# Uma Epidemia de Poisson

Um modelo basico de epidemia numa população de N indivíduos considera eles divididos em quatro grupos:

* S: Suscetíveis (sem imunidade)
* I: Infectados (não hospitalizados)
* H: Infectados Hospitalizados
* R: Recuperados (com imunidade)

De acordo com uma unidade temporal $\delta t$, os números de indivíduos dos grupos acima seguem as seguintes regras:

- Taxa média de novos contágios $r_{I}(t) = \beta(t) * \frac{(I(t) + H(t)) * S(t)}{N}$ por dia.
- Em média, 5% dos novos infectados são levados ao hospital.
- $\beta(t) = \beta(0) = 0.2592$, com $\frac{1}{15}$ de chance de $\beta(t) = 3 \cdot \beta_{0}$ em determinado dia.
- Taxa de recuperação de infectados não hospitalizados $r_{(I)} = 0.07143$ ao dia.
- Taxa de recuperação de infectados hospitalizados $r_{(H)} = 0.03571$ ao dia.


# Parâmetros da simulação:

* $N = 10^{4}$ é o tamanho da população inicial.
* $I(0) = \text{randint}(10, 20)$ (infectados iniciais)
* Repetir os cálculos até que $H(t) = 0$ por três dias seguidos.

# Valores observados

Ao final da simulação, foram observados uma sequência de parâmetros importantes, e realizado operações em cima deles:

* Máximo de hospitalizados simultâneos $\text{max}_{t}[H(t)]$
* O tempo $T_{*}$ até ocorrer o máximo.
* Quanto da população pegou a doença (Recuperados + Hospitalizados + Infectados)
* Quanto da população foi hospitalizada.
* O tempo até a 10° hospitalização, e até a 40° hospitalização ($T_{10}$ e $T_{40}$)

N individuos : 

S susceptiveis

I infectados nao hospitalizados

H infectados hospitalizados 

R recuperados




- 2 dias por mes são excepcionais;

- Nos dias excepcionais $\beta(t) = 3 * \beta(0)$

- Taxa media de recuperacao dos infectados nao hospitalizados $(I)$ eh $r_{RI}(t) =  \gamma_i I(t), 
( γ _i = 0.07143/dia)$ (duracao media da doenca ~= duas semanas)

- Taxa media de recuperacao dos infectados hospitalizados $(H)$ eh $r_{RH}(t) =  \gamma_i H(t)$, 
com $\gamma_i = 0.03571/dia$ (duracao media da doenca ~= quatro semanas)


_____________________________________________________________________________________
SIMULACAO 

N = $10^4$

$I(0)$ = rand(10, 20)

$H(0)$ = 0

$t$ ate nao haver mais pessoas hostpitalizadas por 3 dias seguidos

Queremos salvar as variaveis
- $X = max_t H(t)$
- O tempo T*, desde a primeira hospitalizacao ate o pico maximo de hospitalizados
- a fracao da populacao que teve a doenca uma vez acabada a epidemia. A fracao que esteve hospitalizada
- O tempo desde a primeira hospitalizacao ate a hospitalizacao numero 10, e ate a hopitalizacao 40 ($T_10$ e $T_40$))




O que descobrimos:


- $\beta$ é influenciado por $t$ da seguinte maneira:
A cada passagem de t, existe uma chance de $\frac{1}{15}$ de ser um dia especial, se for o caso, $\beta = 3 * \beta_0$
- Não esquecer q o número de recuperados não ode ser maior q o de novos infectados

In [227]:
from random import random, randint
import copy


import numpy as np
import pandas as pd
from typing import Union
import plotly.express as px


pd.options.plotting.backend = "plotly"

In [2]:
def compare_sign(effective, received):
    if not received:
        return False
    for sign in received:
        if not effective.get(sign):
            continue
        if effective[sign][-1] != received[sign][-1]:
            return False
    return True

In [85]:
def vectorized_min(baseline, array):
    return np.array([min(baseline, x) for x in array])

In [310]:
def avg(array: Union[np.ndarray, pd.DataFrame]) -> Union[np.ndarray, pd.DataFrame]:
    result = sum(array) / len(array)
    if isinstance(result, np.ndarray):
        return result.astype(np.int32)
    else:
        return result.astype('int')

In [151]:
class SimulateEpidemic:
    def __init__(self, starting_values, model=np.random.poisson):
        self.susceptible = starting_values.get("num_susceptible", [int(1e4)])
        self.infected = starting_values.get("num_infected", [20])
        self.hospitalized = starting_values.get("num_hospitalized", [0])
        self.recovered = starting_values.get("num_recovered", [0])
        self.population_size = np.sum(list(starting_values.values()))
        self.population_size = int(self.population_size)

        self.curr_time = 1
        self.model = model

    beta_zero = 0.2592
    gamma_infected = 0.07143
    gamma_hospitalized = 0.03751
    hospitalized_rate = 0.05
    agglomeration_frequency = 1 / 15

    @property
    def keys(self):
        return {"susceptible": self.susceptible, "infected": self.infected,
                "hospitalized": self.hospitalized, "recovered": self.recovered}

    @property
    def df(self):
        return pd.DataFrame.from_dict(self.keys)

    @property
    def beta(self):
        if random() <= self.agglomeration_frequency:
            return self.beta_zero * 3
        else:
            return self.beta_zero

    def update(self, variable, increment, is_new=True, timeframe=-1):
        if is_new:
            next_value = variable[timeframe] + increment
            variable.append(next_value)
        else:
            variable[self.curr_time] += increment
            if variable[self.curr_time] < 0:
                variable[self.curr_time] = 0

    def step(self, recoveries, contagions):
        self.update(self.recovered, sum(recoveries))
        self.update(self.susceptible, -sum(contagions))

        self.update(self.hospitalized, contagions[0])
        self.update(self.hospitalized, -recoveries[0], False)

        self.update(self.infected, contagions[1])
        self.update(self.infected, -recoveries[1], False)

    def apply_model(self, iterable):
        return list(map(self.model, iterable))

    def run(self, remaining_time, stop_sign=None, time_step=1):

        min_count = False
        if stop_sign is None:
            stop_sign = {"day_counter": 0}

        while (remaining_time > 0 or stop_sign["day_counter"] > 0) or not min_count:
            contagions_rate = self.contagions_rate(time_step=time_step)

            avg_contagions = np.array([contagions_rate * self.hospitalized_rate, contagions_rate * (1 - self.hospitalized_rate)])
            avg_recoveries = np.array([self.avg_recovery(is_hospitalized=True), self.avg_recovery()])

            try:
                contagions = self.apply_model(time_step * avg_contagions)
                recoveries = self.apply_model(time_step * avg_recoveries)
            except Exception as e:
                print(e)
                break

            contagions[0] = min(contagions[0], int(self.susceptible[-1] * self.hospitalized_rate))
            contagions[1] = min(contagions[1], int(self.susceptible[-1] * (1 - self.hospitalized_rate)))
            recoveries = vectorized_min(self.infected[-1] + self.hospitalized[-1], recoveries)

            self.step(recoveries, contagions)
            self.curr_time += 1

            if stop_sign["day_counter"] > 0 and compare_sign(self.keys, stop_sign):
                stop_sign["day_counter"] = stop_sign["day_counter"] - 1
            else:
                stop_sign["day_counter"] = stop_sign.get("days_without_hosp", 0)
                remaining_time -= 1

            if contagions[0] != 0:
                min_count = True

    def avg_recovery(self, is_hospitalized=False):
        if is_hospitalized:
            return self.gamma_hospitalized * self.hospitalized[-1]
        else:
            return self.gamma_infected * self.infected[-1]

    def contagions_rate(self, timeframe=-1, time_step=1):
        if not (0 < timeframe < self.curr_time or 0 >= timeframe >= (-1 * self.curr_time)):
            raise ValueError(f"Given timeframe not within acceptable range: [{(-1 * self.curr_time)}, {self.curr_time}], {timeframe}")

        return self.beta * (self.infected[timeframe] + self.hospitalized[timeframe]) * self.susceptible[timeframe] / self.population_size

In [152]:
def reset_default_constants():
    SimulateEpidemic.beta_zero = 0.2592
    SimulateEpidemic.gamma_infected = 0.07143
    SimulateEpidemic.gamma_hospitalized = 0.03751
    SimulateEpidemic.avg_hospitalized = 0.05
    SimulateEpidemic.agglomeration_frequency = 1 / 15

In [354]:
class SimulationGenerator:
    def __init__(self, model=np.random.poisson, starting_values=None, time_step=1):
        self.model = model
        if starting_values is None:
            num_susceptible = int(1e4)
            num_infected = randint(10, 20)
            self.starting_values = {"num_susceptible": [num_susceptible], "num_infected": [num_infected]}
        else:
            self.starting_values = starting_values
        self.time_step = time_step

    @property
    def stop_sign(self):
        return {"hospitalized": [0], "days_without_hosp": np.ceil(3/self.time_step), "day_counter": np.ceil(3/self.time_step)}

    def __next__(self):
        simulation = SimulateEpidemic(copy.deepcopy(self.starting_values), self.model)
        return simulation

    def run(self, simulation=None, return_simulation=False):
        if simulation is None:
            return_simulation = True
            simulation = self.__next__()
        simulation.run(0, self.stop_sign, self.time_step)

        return simulation if return_simulation else None

    def create_single(self):
        simulation = self.__next__()
        return self.run(simulation, return_simulation=True)

    def create_sequence(self, n=10):
        simulations = [self.__next__() for _ in range(n)]
        dataframes = [self.run(sim, return_simulation=True).df for sim in simulations]

        max_length = min([len(df) for df in dataframes])
        dataframes = [df.iloc[:max_length] for df in dataframes]

        return dataframes

    def moving_average(self, sequence=None, n=10):
        if sequence is None:
            sequence = self.create_sequence(n)
        iterations = sequence

        mav = []
        for i in range(len(iterations)):
            mav.append(avg(iterations[:i + 1]))
        return mav

    @staticmethod
    def describe_iteration(simulation: SimulateEpidemic):
        hospitalized_peak = max(simulation.hospitalized)
        peak_index = simulation.hospitalized.index(hospitalized_peak)
        hospitalized_first = np.nonzero(simulation.hospitalized)[0][0]

        print(f"O maximo de hospitalizados é: {hospitalized_peak}")
        print(f"O pico de hospitalizados foi no dia: {peak_index}")
        print(f"T* = {peak_index - hospitalized_first}")

        t_10 = np.where(np.array(simulation.hospitalized) >= 10)[0][0]
        t_40 = np.where(np.array(simulation.hospitalized) >= 40)[0][0]
        print(f"hospitalizados[10] = {t_10}, hospitalizados[40] = {t_40}")

In [355]:
class SimulationViewer:
    def __init__(self, generator=None):
        if generator is None:
            self.generator = SimulationGenerator()
        else:
            self.generator = generator

    def simple_graph(self, dataframe=None):
        if dataframe is None:
            dataframe = self.generator.create_single().df

        color = {"susceptible": "blue", "infected": "red", "hospitalized": "orange", "recovered": "green"}
        fig = dataframe.plot(title="Simulação da Pandemia",
                             template="simple_white",
                             labels=dict(index="dT passados", value="Pessoas", variable="Situação da População"),
                             color_discrete_map=color)
        fig.show()


    def heatmap(self, dataframe=None):
        if dataframe is None:
            dataframe = self.generator.create_single().df

        fig = px.imshow(dataframe.corr(), template="simple_white", text_auto=True)
        fig.show()

In [356]:
viewer = SimulationViewer(SimulationGenerator())

In [357]:
viewer.heatmap()

In [358]:
# sou foda